In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

## Do not forget to adjust filename!

In [ ]:
# read data you want to plot as 2D-image
#data_to_plot_file = '../DATA/RESULTS/KRIGING/filename.xyz'
data_to_plot_file = '../DATA/RESULTS/ML/25_neighbors/filename.xyz'


with open(data_to_plot_file) as file:
    data_to_plot = file.readlines()
    
data_to_plot = [x.strip() for x in data_to_plot] # remove blank and \n
del data_to_plot[0] # remove header

# read the positions of the used stations - overlay in plot
stations_file = '../DATA/00_stations_in_use.csv'
stations = pd.read_csv(stations_file, delimiter=';')
stations_east = stations['east']
stations_north = stations['north']

In [ ]:
# function to create dict-entry for each coordinate
def prepare_dict(data_to_plot):
    kvp = {}
    for line in data_to_plot:
        split = line.split(';')
        key = '{0}{1}'.format(int(float(split[0])), int(float(split[1])))
        value = split[2]
        kvp[key] = value
    return kvp

In [ ]:
# function to fit data into a boundingbox - prepare masking
def create_plottable_matrix(bounds, kvp):
    result = []
    x, y, width  = bounds['minx'], bounds['miny'], bounds['width']
    while y <= bounds['maxy']:
        while x <= bounds['maxx']:
            if '{}{}'.format(x, y) in kvp:
                result.append(float(kvp['{}{}'.format(x, y)]))
            else:
                result.append(-99)
            x += width
        y += width
        x = bounds['minx']
    return result

In [ ]:
# set boundingbox and resolution of data
bounds = {
    'minx':32280000,
    'miny':5577400,
    'maxx':32532000,
    'maxy':5820400,
    'width':200
}

In [ ]:
# create key-values-pairs with coordinates and values, prepare plotable matrix
kvp = prepare_dict(data_to_plot)
data = create_plottable_matrix(bounds, kvp)
data = np.asarray(data) # convert to numpy-array for reshaping
masked_data = np.ma.masked_where(data == -99, data) # masking (transparency)

In [ ]:
# reshape data to prepare rows and cols for plot
x_per_row = int(((bounds['maxx']-bounds['minx'])/200)+1)
y_per_row = int(((bounds['maxy']-bounds['miny'])/200)+1)
grid = masked_data.reshape(y_per_row, x_per_row)

## Adjust Path to result file before run!

In [ ]:
# plot the result 
w = (1*x_per_row)/100
h = (1*y_per_row)/100
fig, ax = plt.subplots(figsize=(w, h))
#ax.set_aspect('equal')


# plot values as image
im = ax.imshow(grid, extent=(bounds['minx'], bounds['maxx'], bounds['maxy'], bounds['miny']), cmap='viridis')
# plot stations as scatterplot + overlay
sc = ax.scatter(stations_east, stations_north, c='salmon', s=4*w)

# Flip picture
ax.set_ylim(ax.get_ylim()[::-1])

# Legend
clb = fig.colorbar(im, ax=ax)
clb.set_label('   [mm]', y=1.001,  rotation='0', size=20)

ax.legend(['Beobachtungsstationen'], loc='upper left')
ax.set_title('Kartendarstellung der interpolierten Niederschlagsverteilung')
# Make visible
fig.tight_layout()
plt.show

plt.savefig('../DATA//RESULTS/ML/25_neighbors/filename.png', dpi=300, format='png')